In [2]:
!pip install mlxtend

     ---------------------------------------- 1.4/1.4 MB 2.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer


from mlxtend.frequent_patterns import apriori

from mlxtend.frequent_patterns import association_rules

## Data Preprocessing

In [15]:
orders = pd.read_csv("Data/orders.csv.zip", nrows=10000)
products = pd.read_csv("Data/products.csv.zip")

In [16]:
#for orders we only use "order_id", "product_id"
orders = orders[["order_id", "product_id"]]
orders.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [17]:
#for products we only use "product_id", "product_name"
products = products[["product_id", "product_name"]]
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [18]:
#now we need to define a dictionary to map product ID to its name.
product_name_map = dict(zip(products["product_id"], products["product_name"]))

In [19]:
#we can merge the products in the same order into a list 
# Group orders by order id and merge them into a list.
order_baskets = orders.groupby("order_id")["product_id"].apply(list)

# Convert the above pandas Series to a pandas DataFrame.
order_baskets = order_baskets.to_frame(name="products_id")

# Create a new column called size that denotes the order sizes.
order_baskets["size"] = order_baskets["products_id"].apply(len)

# Let's have a look at our processed data!
order_baskets.head()

,products_id,size
order_id,,
2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014...",9
3,"[33754, 24838, 17704, 21903, 17668, 46667, 174...",8
4,"[46842, 26434, 39758, 27761, 10054, 21351, 225...",13
5,"[13176, 15005, 47329, 27966, 23909, 48370, 132...",26
6,"[40462, 15873, 41897]",3


In [10]:
#We can use the product_name_map we created just now to figure out what products any given order contain. For example, order 42.

order_42 = order_baskets.loc[42]["products_id"]
for product_id in order_42:
    print(product_name_map[product_id])

Unsweetened Vanilla Almond Breeze
Pure Irish Butter
Bag of Organic Bananas


Next, we are now going to use the mlxtend for frequent itemset mining. This package requires that the itemsets be transformed into a matrix before being passed to its APIs, where each row represents an itemset and each column represents an item. Each cell encodes whether an item is in an itemset or not. You should know what this transformation does after doing the first assignment. Here we implement this transformation with the MultiLabelBinarizer in scikit-learn(sklearn).

In [11]:
mlb = MultiLabelBinarizer()
prod_matrix = pd.DataFrame(data=mlb.fit_transform(order_baskets["products_id"]), index=order_baskets.index, columns=mlb.classes_)
prod_matrix.head()

,1,2,3,4,8,9,10,12,14,15,...,49676,49677,49678,49679,49680,49681,49683,49685,49686,49688
order_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now, let's examine the popularity of individual products, that is, the counts and the distribution of single items (products). The number of orders containing a given product can be calculated as the sum of the column in the product matrix.

In [12]:
prod_popularity = prod_matrix.sum(axis=0)

for prod_id, prod_freq in prod_popularity.head().items():
    print(f"{product_name_map[prod_id]} - {prod_freq}")

Chocolate Sandwich Cookies - 59
All-Seasons Salt - 3
Robust Golden Unsweetened Oolong Tea - 3
Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce - 10
Cut Russet Potatoes Steam N' Mash - 2


## Exercise 1. Find the most popular products

In [13]:
#this function return a list of the  𝑛 most popular products based on the prod_popularity DataFrame.

def top_n_products(n):
    sorted_products = prod_popularity.sort_values(ascending=False)
    top_n_product_ids = sorted_products.head(n).index.tolist()
    top_n_products = [product_name_map[prod_id] for prod_id in top_n_product_ids]
    return top_n_products

In [14]:
# let's test our function to find the top 2 and top 5 products

assert top_n_products(2) == [
    'Banana',
    'Bag of Organic Bananas'
]
assert top_n_products(5) == [
    'Banana',
    'Bag of Organic Bananas',
    'Organic Strawberries',
    'Organic Baby Spinach',
    'Organic Hass Avocado'
]

## Exercise 2. Finding Frequent Itemsets with Apriori

In [20]:
### we limit the number of products in this problem to 100 by popularity 
#so that the apriori algorithm will not run for a long time

data1 = mlb.fit_transform(order_baskets["products_id"]).astype(bool)
prod_matrix1 = pd.DataFrame(
    data=data1, index=order_baskets.index, columns=mlb.classes_
)
prod_popularity1 = prod_matrix1.sum(axis=0)
prod_matrix1 = prod_matrix1.astype(bool)
top_prods = prod_popularity1.sort_values(ascending=False).head(100).index
prod_matrix1 = prod_matrix1[top_prods]
prod_matrix1.head()

,24852,13176,21137,21903,47209,47766,16797,47626,22935,27966,...,41220,5450,5025,16185,39928,23375,7781,21709,27344,31915
order_id,,,,,,,,,,,,,,,,,,,,,
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,True,False,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
prod_matrix1.shape

(977, 100)

In [25]:
def prod_frequent_itemsets(prod_matrix1, min_support=0.005, k=3):
    frequent_itemsets = apriori(prod_matrix1, min_support=min_support, use_colnames=True)
    
    frequent_k_itemsets = frequent_itemsets[frequent_itemsets["itemsets"].apply(lambda x: len(x) == k)]
    
    return frequent_k_itemsets

In [27]:
#now Let's obtain all frequent 3-itemsets with a min support of 0.004 by running the following command.
prod_frequent_3itemsets = prod_frequent_itemsets(prod_matrix1, min_support=0.004, k=3)
#let's test this function
for row in prod_frequent_3itemsets.itertuples():
    assert row.support >= 0.004, f"[Exercise 2] The support of the itemset {row.itemsets} is below the threshold."
    assert len(row.itemsets) == 3, f"[Exercise 2] The itemset {row.itemsets} is not a 3-itemset."


In [28]:
#Let's replace the product IDs with their names.
prod_frequent_3itemsets.itemsets = prod_frequent_3itemsets.itemsets.apply(lambda row: [product_name_map[i] for i in row])
prod_frequent_3itemsets

,support,itemsets
374,0.004094,"[Organic Strawberries, Banana, Organic Baby Sp..."
375,0.004094,"[Banana, Organic Avocado, Organic Baby Spinach]"
376,0.005118,"[Organic Zucchini, Banana, Organic Baby Spinach]"
377,0.004094,"[Organic Large Extra Fancy Fuji Apple, Banana,..."
378,0.005118,"[Boneless Skinless Chicken Breasts, Banana, Or..."
379,0.004094,"[Honeycrisp Apple, Banana, Strawberries]"
380,0.004094,"[Honeycrisp Apple, Banana, Organic Whole Milk]"
381,0.007165,"[Bag of Organic Bananas, Organic Strawberries,..."
382,0.004094,"[Bag of Organic Bananas, Organic Hass Avocado,..."
383,0.004094,"[Raspberries, Hass Avocados, Strawberries]"


## Exercise 3. Apriori Algorithm-Candidate Generation 

A critical step of the Apriori algorithm is candidate generation. That is, candidate (k+1)-itemsets should be generated from frequent k-itemsets. In the following exercise, we want you to generate candidate 3-itemsets based on the frequent 2-itemsets.

In [29]:
def get_2_item_subsets(itemset):
    subsets = []
    items = list(itemset)
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            subsets.append(frozenset([items[i], items[j]]))
    return subsets

def generate_candidate_3_itemsets(base_itemsets, pruning=False):
  
    candidates = set()

    candidate_items = set()
    for itemset in base_itemsets:
        for i in itemset:
            candidate_items.add(i)

    for itemset in base_itemsets:
        pair = list(itemset)
        for item in candidate_items:
            if item not in pair:
                target = frozenset(pair + [item])
                
                if not pruning or all(subset in base_itemsets for subset in get_2_item_subsets(target)):
                    candidates.add(target)
 

    return list(candidates)


In [31]:
# let's test this function on a dataset 
frequent_2_itemsets = []
with open('Data/prod_frequent_2_itemsets.csv', 'r') as fin:
    frequent_2_itemsets.extend({int(x)
                                for x in line.strip().split(",")}
                               for line in fin)

candidate_3_itemsets = generate_candidate_3_itemsets(frequent_2_itemsets, pruning=False)

In [32]:
# assertions
examined = []
for candidate in candidate_3_itemsets:
    assert len(
        candidate) == 3, f"[Exercise 3] Candidate {candidate} not a 3-itemset"
    assert candidate not in examined, f"[Exercise 3] Duplicate 3-itemsets: {candidate}"
    c1, c2, c3 = list(candidate)
    assert {c1, c2} in frequent_2_itemsets or {
        c1, c3
    } in frequent_2_itemsets or {
        c2, c3
    } in frequent_2_itemsets, f"[Exercise 3] Candidate {candidate} does not contain frequent 2-itemsets"

    examined.append(candidate)


## Exercise 4. Apriori Algorithm-Scan Database

In [34]:
#get the dataset ready
# Group orders by order id and merge them into a set.
order_baskets1 = orders.groupby("order_id")["product_id"].apply(frozenset)

# Convert the above pandas Series to a pandas DataFrame.
order_baskets1 = order_baskets1.to_frame(name="products_id")


In [38]:
def calculate_frequent_itemsets(candidate_itemsets, min_support=0.005):
    frequent_itemsets = []
    
    candidate_counts = {candidate: 0 for candidate in candidate_itemsets}
    for basket in order_baskets["products_id"]:
        for candidate in candidate_itemsets:
            if candidate.issubset(basket):
                candidate_counts[candidate] += 1

    # Check and keep only the frequent itemsets with support >= min_support
    total_baskets = len(order_baskets)
    for candidate, count in candidate_counts.items():
        support = count / total_baskets
        if support >= min_support:
            frequent_itemsets.append(candidate)
    return frequent_itemsets

In [39]:
calculate_frequent_itemsets(candidate_3_itemsets)

[frozenset({13176, 21137, 47209}),
 frozenset({21903, 24852, 45007}),
 frozenset({21903, 24852, 25890})]

In [40]:
#assertion

answer = calculate_frequent_itemsets(candidate_3_itemsets)

assert len(answer) == 3
assert {21903, 24852, 45007} in answer


## Exercise 5. Evaluating Frequent Itemsets

In [43]:
# we use the "lift" measurement
prod_matrix1.columns = [product_name_map[x] for x in prod_matrix1.columns]
prod_frequent_itemsets = apriori(prod_matrix1, min_support=0.005, use_colnames=True)

In [44]:
interestingness_measurements = association_rules(prod_frequent_itemsets, metric="lift", min_threshold=0)
interestingness_measurements.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Banana),(Organic Strawberries),0.159672,0.074719,0.012282,0.076923,1.029505,0.000352,1.002388,0.034105
1,(Organic Strawberries),(Banana),0.074719,0.159672,0.012282,0.164384,1.029505,0.000352,1.005638,0.030973
2,(Organic Baby Spinach),(Banana),0.071648,0.159672,0.024565,0.342857,2.147253,0.013125,1.278759,0.575524
3,(Banana),(Organic Baby Spinach),0.159672,0.071648,0.024565,0.153846,2.147253,0.013125,1.097143,0.635810
4,(Organic Hass Avocado),(Banana),0.069601,0.159672,0.008188,0.117647,0.736802,-0.002925,0.952371,-0.277424


In [45]:
#now we are going to implement mutual information measurement
#add a "mutual information" column to the dataframe

def log2(x):
    # Custom log2 function without using math.log2
    if x == 0:
        return float('-inf')
    return np.log(x) / np.log(2)

def mi(antecedent_support, consequent_support, support):
       
    px1 = antecedent_support
    px0 = 1 - antecedent_support
    py1 = consequent_support
    py0 = 1 - consequent_support
    
    px1y1 = support
    px1y0 = px1 - px1y1
    px0y1 = py1 - px1y1
    px0y0 = 1 - px1 - py1 + px1y1
    
    mutual_information = 0
    
  
    
    if px1y1 > 0:
        mutual_information += px1y1 * log2(px1y1 / (px1 * py1))
    if px1y0 > 0:
        mutual_information += px1y0 * log2(px1y0 / (px1 * py0))
    if px0y1 > 0:
        mutual_information += px0y1 * log2(px0y1 / (px0 * py1))
    if px0y0 > 0:
        mutual_information += px0y0 * log2(px0y0 / (px0 * py0))
    
    
    
    return mutual_information

In [46]:
#assertion
assert np.abs(mi(0.6, 0.75, 0.4) - 0.04287484674660057) < 1e-8
assert np.abs(mi(0.5, 0.5, 0.25) - 0) < 1e-8

assert np.abs(mi(0.5, 0.5, 0.5) - 1) < 1e-8

The for loop of Python is very slow. In real-world scenarios, we will use vectorization to speed up the computation. For the mi() function, we can also vectorize it to be mi_vec(). Now, suppose the inputs are all arrays in shape of (n,), how can you implement this function?

Hint:

The returned mutual_information should also be in shape of (n,).
Use np.where() function to replace if-condition. See numpy.where document.

In [47]:
def log2(x):
    
    return np.log(x) / np.log(2)

def mi_vec(antecedent_support: np.ndarray, consequent_support: np.ndarray,
           support: np.ndarray):

    px1 = antecedent_support
    px0 = 1 - antecedent_support
    py1 = consequent_support
    py0 = 1 - consequent_support

    px1y1 = support
    px1y0 = px1 - px1y1
    px0y1 = py1 - px1y1
    px0y0 = 1 - px1 - py1 + px1y1

    mutual_information = np.zeros_like(support)
    
    mutual_information += np.where(px1y1 > 0, px1y1 * log2(px1y1 / (px1 * py1)), 0)
    mutual_information += np.where(px1y0 > 0, px1y0 * log2(px1y0 / (px1 * py0)), 0)
    mutual_information += np.where(px0y1 > 0, px0y1 * log2(px0y1 / (px0 * py1)), 0)
    mutual_information += np.where(px0y0 > 0, px0y0 * log2(px0y0 / (px0 * py0)), 0)

    

    return mutual_information

Mutual information is a classical measure of interestingness. We encourage you to think about the following questions (not graded):

What is the maximum of mutual information in this setting? How to reach it?
What is the minimum of mutual information in this setting? How to reach it?
What does the max/min value imply?

In [48]:
interestingness_measurements['mi'] = \
    interestingness_measurements.apply(lambda pair: mi(pair['antecedent support'], 
                                              pair['consequent support'], 
                                              pair['support']),
                                       axis=1)
interestingness_measurements.sort_values('mi', ascending=False).head(n=5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,mi
261,(Raspberries),(Strawberries),0.026612,0.053224,0.010235,0.384615,7.226331,0.008819,1.538511,0.885174,0.020237
260,(Strawberries),(Raspberries),0.053224,0.026612,0.010235,0.192308,7.226331,0.008819,1.205147,0.910054,0.020237
93,(Bag of Organic Bananas),(Organic Hass Avocado),0.121801,0.069601,0.023541,0.193277,2.776940,0.015064,1.153307,0.728641,0.017598
92,(Organic Hass Avocado),(Bag of Organic Bananas),0.069601,0.121801,0.023541,0.338235,2.776940,0.015064,1.327056,0.687760,0.017598
282,(Organic Garlic),(Organic Zucchini),0.035824,0.029683,0.008188,0.228571,7.700493,0.007125,1.257819,0.902468,0.016409


Let's benchmark the speed of two different implementations. We scale the data by 1000 times. Can you find a significant acceleration in the vectorized version of mi_vec()?

In [49]:
benchmark_data = pd.concat([interestingness_measurements] * 1000,
                           ignore_index=True)
print(f"benchmark_data contains {len(benchmark_data)} entries.")

start = time.time()
result1 = benchmark_data.apply(lambda pair: mi(
    pair['antecedent support'], pair['consequent support'], pair['support']),
                                             axis=1)
end = time.time()
print(f"mi() takes {end-start:.4f} seconds.")

start = time.time()
result2 = mi_vec(benchmark_data['antecedent support'],
                 benchmark_data['consequent support'],
                 benchmark_data['support'])
end = time.time()
print(f"mi_vec() takes {end-start:.4f} seconds.")

assert abs(result1 - result2).max() < 1e-8, "mi_vec() and mi() produce inconsistent results."

benchmark_data contains 332000 entries.
mi() takes 5.5933 seconds.
mi_vec() takes 0.0485 seconds.
